# Testing MultiSigBTC

##### Some imports we need for the testing

In [1]:
import json

##### And some variable

In [2]:
path_john = "output\\john.json"
path_bob = "output\\bob.json"
path_maria = "output\\maria.json"
path_alice = "output\\alice.json"
path_multisig_2_3 = "output\\multisig-2-3.json"
path_multisig_2_3_transaction = "output\\multisig-2-3-transaction.json"
path_multisig_2_2 = "output\\multisig-2-2.json"
path_multisig_2_2_transaction = "output\\multisig-2-2-transaction.json"
path_multisig_2_3_transact_2 = "output\\multisig-2-3-transaction_2.json"

### Update bitcoin.conf

<p>Before continue, you need to run a bitcoin node in regtest and enable server in configuration.<br>
To do so, we need to add the lines bellow in the bitcoin configuration (we use string 'admin' as the rpc use and its password)</p>

>regtest=1<br>
>server=1<br>
>rpcuser=admin<br>
>rpcpassword=admin<br>

**/!\\** *After we update the configuration file, we must execute the command **bitcoind** in a terminal/command line before we continue with the test*

## Prepare our regtest blockchain

Let's create a wallet and mine some blocks to get some bitcoin flowing in our blockchain

In [3]:
!bitcoin-cli createwallet wallet

{
  "name": "wallet",
  "warning": ""
}


In [4]:
miner = !bitcoin-cli getnewaddress address
miner = miner[0]
miner

'bcrt1q5cm4pfhkc645pqynyxx0dhdqvn35qz5xwa9fdc'

In [5]:
!bitcoin-cli generatetoaddress 101 {miner}

[
  "4bb8c47e2ba67faba691729a786fe68c750fba9ac9649bb3bb36c4cafbf7b5a2",
  "5a10dcf36e87225952ec7db9a9a5d5273e7617dce0f04678ea0f4f5e77d1c954",
  "304b044af4876654a4512213cd1267f14530212f8f9f221ef69cc9f54e0132d9",
  "6a848c0f4d04be2d4a62c39af5c486a3ee8ce5d21473791c9ffd6b9fc3c3c5a2",
  "3531150835018db694e46b29a64293dfcf065b505e2f6f8e58af50343e7445d0",
  "0139fc5e3bd7e4c6794f60a7ff3449171d431404a9ad864fdc2378b35f6307e4",
  "2a05ae1ea45d484648c6b18eb20517647ea17a3b6164d45cd79991bf310a546d",
  "5d394bcc9d1de0cb70112d338cbc2bbec5a064068191921a191ae00b24c62865",
  "4a81d0f802ead8a4921808733743ec63ff40bfeef8a3ef5b79d681ecc5442e39",
  "531a77f9ffc052bcdf579cf069d0072c59f6779d38305bdcdc6a1e10135cefc1",
  "5d4bec96f99c57d80dc70aa76e9bcc4c76c7ccc2038db44e8fbb81c2433d0ac1",
  "677c536d929f4742c89ba2262bc8c8d4d68afd7f3166c9f1811d924cc03db848",
  "729a386a046e0a2cf85b2ed53c76da80b66eca5afa32f6094dc58be548b0b3f8",
  "7bca3b2db032284f335b1f24695d52220fa650ec09dc8b45909cdacb7915eb18",
  "4dac22cbd295e07

In [6]:
!bitcoin-cli listaddressgroupings

[
  [
    [
      "bcrt1q5cm4pfhkc645pqynyxx0dhdqvn35qz5xwa9fdc",
      50.00000000,
      "address"
    ]
  ]
]


Ok so we created a wallet and an address and mined 101 blocks in order to get some funds in our address to work with (miner reward is locked for 100 blocks)

## Create some private-public key pairs to work with

##### John's Keys

In [7]:
result = !python.exe MultiSigBTC.py -o {path_john} create_new_address "John" "pass" -s "john"
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"')
result = json.loads(result)
john = result['address']
result

{'address': 'bcrt1qkx5vdrj0jz73n8qkssfptfanknnh4gsey7qr6n',
 'keys': {'name': 'John',
  'private': '+wGTOevODdDFajX65ZsYn+dk2KIx4XmpeDnUrIoK+pPZI/sVlp8T4J3pbAn30zmjgNTtYfF/uXBiiHNqNK4RtfduKdK2ACezsTwFK4W6tWHPJ0q/cWogP4GTOWQmW5PoHF6J7NPtyfs=',
  'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
 'type': 'address'}

##### Bob's Keys

In [8]:
!python.exe MultiSigBTC.py -o {path_bob} create_new_address "Bob" "pass" -s "bob" -t "legacy"

{'address': 'msXtCoKiBft566TaYP9xjMAbRe1dPdSrSj',
 'keys': {'name': 'Bob',
          'private': '8m5Kn3HPvd+kAjnSeC+8ioRs6hjar8taTABkDk+oRRQjCxudZ1ispqwMp2TmBSNDMDjNelPVoiD8KtfUtMkN7GeZQub2bv8Ba0e59jqAi6Q2zLmp0kJN16nbuiUfaFACJLJvqz7jQPk=',
          'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
 'type': 'address'}


##### Maria's Keys

In [9]:
!python.exe MultiSigBTC.py -o {path_maria} create_new_address "Maria" "pass" -s "maria"

{'address': 'bcrt1qvxkn6zzxtgulg2mscsv48468473jjldgra0ha7',
 'keys': {'name': 'Maria',
          'private': 'biLKoGd77Hzq60JqbSKeFLV2lgzJcTr0n8n9xpe1urUNi0W/hv+ieZjXipTEMNQe+cm2GDMBrZ/wGnbUe/xukY5UhhpOHyYuI8zKH9WKAaDqD8W6M64QX/8jvxFYI6kYk05PFN1jbSU=',
          'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
 'type': 'address'}


##### Alice's Keys

In [10]:
!python.exe MultiSigBTC.py -o {path_alice} create_new_address "Alice" "pass" -s "alice"

{'address': 'bcrt1q8vwf9rfkrzwmj7hw3dre4d37aw440jsw3pkz02',
 'keys': {'name': 'Alice',
          'private': 'mZz0ZhnFhSUzGECDHLNOi7eMuBcMTLnxlRl7gB0dDXfzLEtiVn5tsdQPOzaxhBRtpeReAUbL5vs5UHJ12sx2v180x8F69tlfuMnU9YWFvdMWEaOhBCnfW+IoKoascfVIyphdUblZZA4=',
          'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'},
 'type': 'address'}


## Create a 2 out of 3 Multi Signature Address

In [11]:
result = !python MultiSigBTC.py -o {path_multisig_2_3} create_multisig_address 2 {path_john} {path_bob} {path_alice} -t "legacy"
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
multisig_2_3 = result['address']
result

{'address': '2N8ZjuyrxVVHNDfLUKAQshJ4g2bvNkCMaeE',
 'keys': [{'name': 'John',
   'private': '',
   'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
  {'name': 'Bob',
   'private': '',
   'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
  {'name': 'Alice',
   'private': '',
   'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
  '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
  '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
  '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
  3,
  'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_address'}

Now we have our 2 out of 3 multi signature address.

### Send some funds to the newly created multi signature address

In [12]:
!bitcoin-cli sendtoaddress {multisig_2_3} 1

511a994f05c7ff0ad94056a9f116106075f1eef4f54bb35e87033bd8c4c9ca68


In [13]:
!bitcoin-cli sendtoaddress {multisig_2_3} 0.3

a265208c4ef799bfa032e515282e42533aa3051a40c723dd7ecb4f46cec380cb


In [14]:
!bitcoin-cli sendtoaddress {multisig_2_3} 1.1

562e4854b654493eced82541fcd4516c91ecfdd4114a9b62b203e9c70c786855


In [15]:
!bitcoin-cli sendtoaddress {multisig_2_3} 5

26cf3c83ab4131479bd7eb9f1351792cad6aac4377ed5dc86f9b4201b85dfd4c


In [16]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "12befd7e3860d7dc9350d92c50ded07592654ea9906f7b3a2dbfae5a8f0516a1"
]


In [17]:
!bitcoin-cli sendtoaddress {multisig_2_3} 52

77b55ea76c1ab3f06dabc6f2c8680208805bc2dd753ceff2a59b067d48babc4e


In [18]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "62d4d0ee67cb6e55c956bc32393ae837dc4150a4e90ef7688d032fd915f7257c"
]


To make it a little more challenging for our script (MultiSigBTC.py), we send multiple transactions in order for our multi signature address to own multiple UTXOs

### Let's check the balance of our address now

In [19]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 52.0,
            'txid': '77b55ea76c1ab3f06dabc6f2c8680208805bc2dd753ceff2a59b067d48babc4e',
            'vout': 1},
           {'amount': 5.0,
            'txid': '26cf3c83ab4131479bd7eb9f1351792cad6aac4377ed5dc86f9b4201b85dfd4c',
            'vout': 0},
           {'amount': 1.1,
            'txid': '562e4854b654493eced82541fcd4516c91ecfdd4114a9b62b203e9c70c786855',
            'vout': 0},
           {'amount': 1.0,
            'txid': '511a994f05c7ff0ad94056a9f116106075f1eef4f54bb35e87033bd8c4c9ca68',
            'vout': 1},
           {'amount': 0.3,
            'txid': 'a265208c4ef799bfa032e515282e42533aa3051a40c723dd7ecb4f46cec380cb',
            'vout': 1}],
 'balance': 59.4,
 'type': 'balance'}


### Now that we have some funds, let's create a transaction

We will try to send 58 BTC to John's address in order to control alone those 58 BTC, with his keys

In [20]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transaction} create_multisig_transaction {john} 58 0.01 {path_multisig_2_3}

{'UTXOs': [{'amount': 52.0,
            'txid': '77b55ea76c1ab3f06dabc6f2c8680208805bc2dd753ceff2a59b067d48babc4e',
            'vout': 1},
           {'amount': 5.0,
            'txid': '26cf3c83ab4131479bd7eb9f1351792cad6aac4377ed5dc86f9b4201b85dfd4c',
            'vout': 0},
           {'amount': 1.1,
            'txid': '562e4854b654493eced82541fcd4516c91ecfdd4114a9b62b203e9c70c786855',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b

### Let's try to sign the transaction two of the owners

In [22]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} sign_multisig_transaction {path_multisig_2_3_transaction} {path_bob} "pass"

{'UTXOs': [{'amount': 52.0,
            'txid': '77b55ea76c1ab3f06dabc6f2c8680208805bc2dd753ceff2a59b067d48babc4e',
            'vout': 1},
           {'amount': 5.0,
            'txid': '26cf3c83ab4131479bd7eb9f1351792cad6aac4377ed5dc86f9b4201b85dfd4c',
            'vout': 0},
           {'amount': 1.1,
            'txid': '562e4854b654493eced82541fcd4516c91ecfdd4114a9b62b203e9c70c786855',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b

##### But what will happen if we try to send the transaction partially signed? Let's try it, why not...

In [23]:
!python MultiSigBTC.py get_signed_multisig_transaction {path_multisig_2_3_transaction}

{'exception': {'message': 'The transaction contains only 1 out of 2 signatures '
                          'required',
               'type': 'Exception'},
 'trace': [{'line': 193, 'name': '<module>'},
           {'line': 277, 'name': 'createdSignedTransaction'}],
 'type': 'exception'}


Ah yes I forgot about that, I have implemented a exemption when someone tries to get the signed transaction without having enough signatures... Ok I guess we will not send a partially signed transaction to the network :D

##### Ok let's sign the transaction also with Johns keys then

In [24]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} sign_multisig_transaction {path_multisig_2_3_transaction} {path_john} "pass"

{'UTXOs': [{'amount': 52.0,
            'txid': '77b55ea76c1ab3f06dabc6f2c8680208805bc2dd753ceff2a59b067d48babc4e',
            'vout': 1},
           {'amount': 5.0,
            'txid': '26cf3c83ab4131479bd7eb9f1351792cad6aac4377ed5dc86f9b4201b85dfd4c',
            'vout': 0},
           {'amount': 1.1,
            'txid': '562e4854b654493eced82541fcd4516c91ecfdd4114a9b62b203e9c70c786855',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b

We have now enough keys to created a signed transaction. Please note that it doesn’t really make sense to sign the transaction also with Alice’s key.<br>
MultiSigBTC is just keeping the signatures in the results and it will use just 2 out of 3 no matter if we provide 3 of them. Which 2 you ask?<br>
The first two. Of course this is not the best solution because one of the first two might be invalid in which case it will create an invalid transaction no matter if the third one is fine.<br>
This can be resolved by checking the validity of the signatures before use but let’s just say this part is work in progress :D

### Let's produce the signed transaction and submit it to the network

In [25]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} get_signed_multisig_transaction {path_multisig_2_3_transaction}

02000000034ebcba487d069ba5f2ef3c75ddc25b80080268c8f2c6ab6df0b31a6ca75eb57701000000fdfd000048304502210096dda07ef7c659aab69512359808f08bec71ed15d03155d25a1d7e9dfb91d68e0220183fa1551c253509b0bfaca7ab2433a99861d325d60690021132856411cc62a901473044022018efbfe2c139e4294c712692e7896cc61d3e0b3c814260646f66881eacca894b022071e1c46be08f00c4bd8aa7eea8d58a9577a9e186351304715b8e408c73ccf301014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff4cfd5db801429b6fc85ded7743ac6aad2c7951139febd79b473141ab833ccf2600000000fdfe0000483045022100a65009c3118aad9868b6b80a0eb1e4bdf71de15e3fbb301ba6cd237623d9a746022006c575b514fe8cf7864093b89a8f267afb9a1d831f3062941cd9ff53bd99445301483045022100d0a93c40d12dcd7949881c6899646eb5e78f5340087496123c21519751eff96902204d979ad865f288bed2b886a8a696e1bf79d9254db6e150ebf7fa9158954de3a1014c695221030a10de3e8ea71824

In [26]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transaction}

{'txid': ['5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9'],
 'type': 'txid'}


**Success!** The transaction has been submitted! We will generate a block now so it can be included in the blockchain and check the new balances

In [27]:
result = !bitcoin-cli generatetoaddress 1 {miner}
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
block = result[0]
block

'328d7206a086ee9b2e385d968e52b50c9910fa1767f37e7d365a3e78bbb7666d'

In [28]:
!bitcoin-cli getblock {block}

{
  "hash": "328d7206a086ee9b2e385d968e52b50c9910fa1767f37e7d365a3e78bbb7666d",
  "confirmations": 1,
  "height": 104,
  "version": 536870912,
  "versionHex": "20000000",
  "merkleroot": "32c5ad6856d358776a055ee24d8bee5d5cb8ceb6ef49ea09c00e2de6c38ae90b",
  "time": 1650370482,
  "mediantime": 1650370395,
  "nonce": 1,
  "bits": "207fffff",
  "difficulty": 4.656542373906925e-10,
  "chainwork": "00000000000000000000000000000000000000000000000000000000000000d2",
  "nTx": 2,
  "previousblockhash": "62d4d0ee67cb6e55c956bc32393ae837dc4150a4e90ef7688d032fd915f7257c",
  "strippedsize": 1176,
  "size": 1212,
  "weight": 4740,
  "tx": [
    "eb2c2a7d0604fc0ca552782b6032a21efe4999766e451f3f69baea54285f857d",
    "5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9"
  ]
}


##### What is the balance of our multisig address now?

In [29]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 1.0,
            'txid': '511a994f05c7ff0ad94056a9f116106075f1eef4f54bb35e87033bd8c4c9ca68',
            'vout': 1},
           {'amount': 0.3,
            'txid': 'a265208c4ef799bfa032e515282e42533aa3051a40c723dd7ecb4f46cec380cb',
            'vout': 1},
           {'amount': 0.09,
            'txid': '5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9',
            'vout': 1}],
 'balance': 1.39,
 'type': 'balance'}


It seems correct! The funds have been subtracted from the balance

##### And what about the balance on John's address we send the funds to?

In [30]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {john}

{'UTXOs': [{'amount': 58.0,
            'txid': '5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9',
            'vout': 0}],
 'balance': 58.0,
 'type': 'balance'}


Everything looks into place! **The test was a success!**

## Create a 2 out of 2 multi signature address

Now that we have some thing already up and working from our previous test, let's try to create another address that needs 3 out of the 4 owners to sign the transactions

### Create the multi signature address

In [31]:
result = !python MultiSigBTC.py -o {path_multisig_2_2} create_multisig_address 2 {path_bob} {path_maria}
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
multisig_2_2 = result['address']
result

{'address': 'bcrt1qvgj2u6es30wnnegwtr4vdr63cahee4qpaxx7ytg488u6s835j9psr637ls',
 'keys': [{'name': 'Maria',
   'private': '',
   'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
  {'name': 'Bob',
   'private': '',
   'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
  '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
  '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
  2,
  'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_address'}

### Send some funds to the newly created address

In [32]:
!bitcoin-cli sendtoaddress {multisig_2_2} 14

62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d


In [33]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "2cfe9542431fb53dc0554ddf59d396510e888010239a6f12fc587785f14f2574"
]


In [34]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_2}

{'UTXOs': [{'amount': 14.0,
            'txid': '62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d',
            'vout': 0}],
 'balance': 14.0,
 'type': 'balance'}


### We will now send some funds to the other multi signature address we created earlier

##### Firstly let's try sending more funds than we have to see what happens

In [35]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_2_transaction} create_multisig_transaction {path_multisig_2_3} 15 0.01 {path_multisig_2_2}

{'exception': {'message': 'Not enough funds. Total amount is 14.0',
               'type': 'Exception'},
 'trace': [{'line': 176, 'name': '<module>'},
           {'line': 134, 'name': 'createMultisigTransaction'},
           {'line': 112, 'name': 'createTransaction'}],
 'type': 'exception'}


Great! The script works as expected, it gets the balance and return an error when we try to send more than we can.

##### Now we create a valid transaction and we will try to spend the whole UTXO we have (meaning no UTXO returning funds to our address)

In [36]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_2_transaction} create_multisig_transaction {path_multisig_2_3} 13.99 0.01 {path_multisig_2_2}

{'UTXOs': [{'amount': 14.0,
            'txid': '62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d',
            'vout': 0}],
 'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'segwit': True,
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '02000000012d9af8b2e725abf8af943266096980c61dcd62fc2cbdee7cbaa30a92347c22620000000000ffffffff01c00b63530000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000'}


So far so good! :D

### Now it's time to sign the transaction

In [37]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_bob} "pass"

{'UTXOs': [{'amount': 14.0,
            'txid': '62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d',
            'vout': 0}],
 'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'segwit': True,
 'signatures': {'62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d': [{'name': 'Bob',
                                                                                      'signature': '3045022100f922b1046c9fe4654752494e3eba5e897fee75f31c1d9648a497935709338cf9022070294ad0b6287c341f02227e01c964fa63fa1e78b

In [38]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_maria} "passs"

{'exception': {'message': 'Failed to decrypt the private key',
               'type': 'Exception'},
 'trace': [{'line': 185, 'name': '<module>'},
           {'line': 207, 'name': 'signMultisigTransaction'},
           {'line': 275, 'name': 'getPrivateKey'}],
 'type': 'exception'}


Ops, wrong password... Did I do it on purpose? Maybe :D

In [39]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_maria} "pass"

{'UTXOs': [{'amount': 14.0,
            'txid': '62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d',
            'vout': 0}],
 'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'segwit': True,
 'signatures': {'62227c34920aa3ba7ceebd2cfc62cd1dc6806909663294aff8ab25e7b2f89a2d': [{'name': 'Maria',
                                                                                      'signature': '3045022100e11152125b464ba3289d97cdbfaf91e83d75c151d6026614968375cd9f457268022073d1395aa1db4427c15fb23b16041208c6a10bf

### Finally let's send the transaction and check the balances

In [40]:
tx = !python MultiSigBTC.py get_signed_multisig_transaction {path_multisig_2_2_transaction}
tx[0]

'020000000001012d9af8b2e725abf8af943266096980c61dcd62fc2cbdee7cbaa30a92347c22620000000000ffffffff01c00b63530000000017a914a80a5ad975825e6ea42dbd841a26c753851810d5870400483045022100e11152125b464ba3289d97cdbfaf91e83d75c151d6026614968375cd9f457268022073d1395aa1db4427c15fb23b16041208c6a10bf3713f838222275fa33e9c3c0b01483045022100f922b1046c9fe4654752494e3eba5e897fee75f31c1d9648a497935709338cf9022070294ad0b6287c341f02227e01c964fa63fa1e78b57a342a4263f5b510f309f30147522102b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c52ae00000000'

In [41]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {tx[0]}

{'txid': ['e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f'],
 'type': 'txid'}


Perfect the transaction has gone through!

### Let's put the transaction in a block and check the new balances

In [42]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "72592034aadcb3766fd183eca2ca0b24b4ff75272399de169aa66e3594360024"
]


In [43]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_2}

{'balance': 0.0, 'type': 'balance'}


In [44]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0},
           {'amount': 1.0,
            'txid': '511a994f05c7ff0ad94056a9f116106075f1eef4f54bb35e87033bd8c4c9ca68',
            'vout': 1},
           {'amount': 0.3,
            'txid': 'a265208c4ef799bfa032e515282e42533aa3051a40c723dd7ecb4f46cec380cb',
            'vout': 1},
           {'amount': 0.09,
            'txid': '5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9',
            'vout': 1}],
 'balance': 15.38,
 'type': 'balance'}


Great! Everything looks into place!

## Try to send a transaction with wrong signatures

Ok now let's try to send some funds from the 2 out of 3 multi signature address to the 2 out of 2 multi signature address

### Let's create the transaction

In [45]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} create_multisig_transaction {multisig_2_2} 10 0.024 {path_multisig_2_3}

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '020

### And now sign it

In [46]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_john} "pass"

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures': {'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f': [{'na

And now we will try to sign with Maria's keys who is not part of the multi signature address

In [47]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_maria} "pass"

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures': {'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f': [{'na

### We are now ready to send it to the network and see what happens

In [48]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} get_signed_multisig_transaction {path_multisig_2_3_transact_2}

02000000019fa3ea57fc11b5b09a8c57e27983cb832372a86c1527b6d7afce941a0c040be500000000fdfd0000483045022100aed33eac16cc51017aae7b98b3d0bc70bbc1ed43ea02e4a1cd53338accf7a1eb02200223052de9003c0b82aee8a3894d8b8960800f0dab04373a43c281b2feb5a8030147304402204b63487dcc57abffe13d21874fb9d78b708b59aa4a92b9dfbbc2b925381203b102206253ccdabb58af06f686492774413bd819c98f9c6549fc8fc800f7a7f1c0d1d8014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff0200ca9a3b000000002200206224ae6b308bdd39e50e58eac68f51c76f9cd401e98de22d1539f9a81e349143c0a2a3170000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000


In [49]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transact_2}

{'exception': {'message': '-26: mandatory-script-verify-flag-failed (Signature '
                          'must be zero for failed CHECK(MULTI)SIG operation)',
               'type': 'JSONRPCException'},
 'trace': [{'line': 207, 'name': '<module>'},
           {'line': 246, 'name': 'sendTransaction'},
           {'line': 262, 'name': 'sendTransaction'},
           {'line': 170, 'name': 'batch_'}],
 'type': 'exception'}


As expected the transaction failed to be submited to the blockchain because the script contains an invalid signature. So all good!

### Let's try to create again the transaction and sign it correctly

In [50]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} create_multisig_transaction {multisig_2_2} 10 0.024 {path_multisig_2_3}

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '020

In [51]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_alice} "pass"

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures': {'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f': [{'na

In [52]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_bob} "pass"

{'UTXOs': [{'amount': 13.99,
            'txid': 'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f',
            'vout': 0}],
 'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'segwit': False,
 'signatures': {'e50b040c1a94ceafd7b627156ca8722383cb8379e2578c9ab0b511fc57eaa39f': [{'na

In [53]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} get_signed_multisig_transaction {path_multisig_2_3_transact_2}

02000000019fa3ea57fc11b5b09a8c57e27983cb832372a86c1527b6d7afce941a0c040be500000000fdfd000048304502210080f12e95aa88c4eabacfb3c3a3a9ea6c5ad70af7d7fe88893adf65309898f07a02205b6271c216b660d63f8a2db051e2e460cb515edad8c1e0f7d692dc181a1bd00a01473044022045ec1bfb4c09cc60e2c952a47ee4674758f514bc235ddd464076164eef5fdf11022062a615b6fb6bb755dfb803c5161852182e27b0a0c2dc5d5f891ba1e44febca96014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff0200ca9a3b000000002200206224ae6b308bdd39e50e58eac68f51c76f9cd401e98de22d1539f9a81e349143c0a2a3170000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000


In [54]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transact_2}

{'txid': ['b35260aa1d68f87417ed0044cf86af16599097181bd1b4821de9f077787aa97e'],
 'type': 'txid'}


In [55]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "21b134cce4deee12c8fe03f5c4042116405cdb270dc34caa12b451fd72768f54"
]


In [56]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {multisig_2_2}

{'UTXOs': [{'amount': 10.0,
            'txid': 'b35260aa1d68f87417ed0044cf86af16599097181bd1b4821de9f077787aa97e',
            'vout': 0}],
 'balance': 10.0,
 'type': 'balance'}


In [57]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {multisig_2_3}

{'UTXOs': [{'amount': 3.966,
            'txid': 'b35260aa1d68f87417ed0044cf86af16599097181bd1b4821de9f077787aa97e',
            'vout': 1},
           {'amount': 1.0,
            'txid': '511a994f05c7ff0ad94056a9f116106075f1eef4f54bb35e87033bd8c4c9ca68',
            'vout': 1},
           {'amount': 0.3,
            'txid': 'a265208c4ef799bfa032e515282e42533aa3051a40c723dd7ecb4f46cec380cb',
            'vout': 1},
           {'amount': 0.09,
            'txid': '5437fd2e7e04c31321489fd74e3c391407dabc7032348a150e0791c0f237eac9',
            'vout': 1}],
 'balance': 5.356,
 'type': 'balance'}


**Perfect!** The test concludes here! :)